<a href="https://colab.research.google.com/github/jkordonis/TropicalFactorization/blob/main/MovieLensTropFactorization_IMPLICIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#import cupy as np
import numpy as np


In [ ]:
# @title Load the MovieLens data (run this cell).

# Download MovieLens data.
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')





In [ ]:
indices_pd = ratings[['user_id', 'movie_id']].values
IND=np.array(indices_pd).astype(int)
IND_x,IND_y = IND[:,0],IND[:,1]

n = 944#(np.max(IND,0)[0]+1).astype(int)
p = 1683# (np.max(IND,0)[1]+1).astype(int)
r = 100
B=np.zeros((n,p))+1
m=20

B[IND_x,IND_y]=-1

In [ ]:
pairs=[(i,j) for j in range(p) for i in range(n)]
pairs=np.array(pairs)
np.random.shuffle(pairs)

N=n*p#1271000 #n*p*0.8
N_=n*p#1429876
N__=n*p


In [ ]:
IND_x_train = pairs[:N,0]
IND_y_train = pairs[:N,1]


IND_x_val  = pairs[N:N_,0]
IND_y_val  = pairs[N:N_,1]

IND_x_tst  = pairs[N_:N__,0]
IND_y_tst  = pairs[N_:N__,1]





B_sparse_ob_or = -B[IND_x_train,IND_y_train]
B_sparse_ob_or_val = -B[IND_x_val,IND_y_val]
B_sparse_ob_or_tst = -B[IND_x_tst,IND_y_tst]


In [ ]:
 # Initialize C,D matrix estimates
C,D_Tr=np.random.rand(n,r),np.random.rand(p,r)


In [ ]:
import time
T=time.time()
cnt=0
Err =np.zeros(100000)
Err_val =np.zeros(100000)
Err_tst =np.zeros(100000)


for _ in range(100):
  for _ in range(10):
    INDICES=np.array(range(N))
    np.random.shuffle(INDICES)
    BatchSize=1000
    i=0
    minibatches=[]
    while i*BatchSize<=N:
      minibatches.append(INDICES[i*BatchSize:BatchSize*(i+1)])
      i+=1

    #minibatches = [INDICES[i*BatchSize:BatchSize*(i+1)] for i in range(np.ceil(N/BatchSize).astype(int))]
    for AA in minibatches:
      IND_x_ob=IND_x_train[AA]
      IND_y_ob=IND_y_train[AA]
      B_sparse_ob=B_sparse_ob_or[AA]

      Maximizers = np.argmax(C[IND_x_ob]+D_Tr[IND_y_ob],-1)
      B_sparse_ob_est = C[IND_x_ob,Maximizers]+D_Tr[IND_y_ob,Maximizers]
      Differences = B_sparse_ob_est -B_sparse_ob
    #  for i in range(IND_x_ob.shape[0]):
      C_to_change = C[IND_x_ob,Maximizers]
      D_to_change = D_Tr[IND_y_ob,Maximizers]

      C[IND_x_ob,Maximizers]=C_to_change-0.03*Differences
      D_Tr[IND_y_ob,Maximizers]=D_to_change-0.03*Differences


    Maximizers = np.argmax(C[IND_x_train]+D_Tr[IND_y_train],-1)
    B_sparse_ob_est = C[IND_x_train,Maximizers]+D_Tr[IND_y_train,Maximizers]
    B_sparse_ob=B_sparse_ob_or[range(N)]
    Differences = B_sparse_ob_est -B_sparse_ob
    Err[cnt]=np.linalg.norm(Differences)
    C=C+0.00031
    D_Tr=D_Tr+0.00031

    Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
    B_sparse_ob_est = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
    Differences_tst = B_sparse_ob_est -B_sparse_ob_or_tst

    Maximizers = np.argmax(C[IND_x_val]+D_Tr[IND_y_val],-1)
    B_sparse_ob_est = C[IND_x_val,Maximizers]+D_Tr[IND_y_val,Maximizers]
    Differences_val = B_sparse_ob_est -B_sparse_ob_or_val
    Err_val[cnt]=np.linalg.norm(Differences_val)
    Err_tst[cnt]=np.linalg.norm(Differences_tst)

    cnt+=1
    U, s, Vt = np.linalg.svd(D_Tr, full_matrices=False)
    Diag_sk_sqrt=np.diag(s[:m]**0.5)
    Uk = U[:, :m]@Diag_sk_sqrt
    Vk = Diag_sk_sqrt@Vt[:m, :]
    #D_Tr = Uk@Vk





  print(time.time()-T,Err[cnt-1],np.linalg.norm(Differences_val),np.linalg.norm(Differences_tst))

31.000489234924316 1048.0890911772674 377.44759810098884 380.97183161319964
46.77662444114685 1008.834545322961 368.5084188495278 372.68076187715786
61.71968126296997 987.0507783033489 365.1953935195755 370.3116837366219
76.13250136375427 972.9151844276824 364.25970385672423 368.8280508821547
90.8269476890564 961.138630451852 363.7581393028738 368.3128678496506
105.41195344924927 952.8610230294909 363.20810620745215 368.17042056901937
120.2000162601471 945.8648011758834 363.01193042556883 368.7266508210252
134.950448513031 940.1639447630741 363.6530462072495 369.2153894040783
150.0908646583557 934.9869131422779 363.8090007730463 369.27695652209746
166.44985055923462 930.7311145348609 363.7428896684168 369.83907133678923
182.23003578186035 926.544386919995 364.9515409956742 369.798850357272


KeyboardInterrupt: ignored

In [ ]:
indexmin=np.argmin(    Err_val[:cnt])
print(Err_tst[indexmin]/((N_-N)**0.5))

In [ ]:
import matplotlib.pyplot as plt
import numpy

#plt.plot(np.asnumpy(Err[:cnt]))
plt.plot(np.asnumpy(Err_tst[20:cnt]))
plt.plot(np.asnumpy(Err_val[20:cnt]))
plt.grid()
#plt.plot(Err[:cnt])